# 챕터 1: Agent Skills 기본 구현

## 1-1. 개요 및 환경 설정

Agent Skills는 Claude에게 특정 도메인의 전문 지식이나 작업 방식을 가르치는 시스템입니다. Skills를 통해 Claude는 일관된 품질의 결과물을 생성할 수 있습니다.

In [ ]:
# 필요한 라이브러리 설치
%pip install anthropic python-dotenv pyyaml

In [1]:
from dotenv import load_dotenv
import os

# .env 파일에 key를 저장하고 환경 변수를 로드한다
!echo "ANTHROPIC_API_KEY=<sk-ant-로 시작하는 발급받은 ANTHROPIC API KEY 붙혀넣기" >> .env
!echo "OPENAI_API_KEY=sk-proj-로 시작하는 발급받은 ANTHROPIC API KEY 붙혀넣기" >> .env
load_dotenv()

True

In [10]:
# 라이브러리 임포트
import anthropic
import os
from pathlib import Path
import yaml
from typing import Dict, List, Optional
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()
MODEL = "claude-sonnet-4-5"

# Anthropic API 클라이언트 초기화
client = anthropic.Anthropic()  # 환경 변수에서 API 키를 가져온다

print("환경 설정 완료")

환경 설정 완료


## 1-2. 직접 읽기 방식 (Direct Reading Method)

가장 간단한 방식으로, skill 내용을 직접 문자열로 작성하여 Claude에게 전달하는 방법입니다.

In [4]:
# Skill 내용을 직접 문자열로 정의한다
BLOG_WRITING_SKILL = """
# 블로그 작성 전문가 Skill

## 목적
고품질의 기술 블로그 포스트를 작성한다.

## 작성 원칙
1. 독자 친화적인 설명을 제공한다
2. 실용적인 예제를 포함한다
3. 명확한 구조를 유지한다
4. 전문 용어는 쉽게 풀어 설명한다

## 구조
- 도입부: 문제 정의 및 배경
- 본문: 상세 설명과 예제
- 결론: 핵심 요약 및 다음 단계

## 스타일
- 문장은 ~다 체를 사용한다
- 능동적이고 간결한 표현을 선호한다
- 코드 블록은 언어를 명시한다
"""

In [5]:
def call_claude_with_skill(user_message: str, skill_content: str) -> str:
    """
    Skill을 포함하여 Claude API를 호출한다
    
    Args:
        user_message: 사용자의 요청 메시지
        skill_content: 적용할 skill의 내용
    
    Returns:
        Claude의 응답 텍스트
    """
    # System prompt에 skill 내용을 포함시킨다
    response = client.messages.create(
        model=MODEL, # 원본의 'claude-sonnet-4-20250514'는 임의의 모델명일 수 있어, 현재 사용 가능한 모델로 변경합니다.
        max_tokens=2000,
        system=skill_content,  # Skill을 시스템 프롬프트로 전달한다
        messages=[
            {
                "role": "user",
                "content": user_message
            }
        ]
    )
    
    return response.content[0].text

In [6]:
# 테스트: Skill을 적용하여 블로그 포스트 작성 요청
user_request = "Python 리스트 컴프리헨션에 대한 블로그 포스트를 작성해줘"
result = call_claude_with_skill(user_request, BLOG_WRITING_SKILL)

print("=" * 80)
print("생성된 블로그 포스트:")
print("=" * 80)
print(result)

생성된 블로그 포스트:
# Python 리스트 컴프리헨션: 간결하고 우아한 리스트 생성법

## 도입부: 반복문의 불편함을 해결하다

Python으로 프로그래밍을 하다 보면 새로운 리스트를 만들기 위해 반복문을 자주 사용한다. 예를 들어, 1부터 10까지의 제곱수를 담은 리스트를 만들려면 다음과 같이 작성한다.

```python
squares = []
for i in range(1, 11):
    squares.append(i ** 2)
print(squares)  # [1, 4, 9, 16, 25, 36, 49, 64, 81, 100]
```

이 코드는 잘 작동하지만, 5줄이나 되고 빈 리스트를 먼저 만들어야 하는 번거로움이 있다. Python은 이런 패턴을 한 줄로 간결하게 표현할 수 있는 **리스트 컴프리헨션(List Comprehension)**이라는 강력한 기능을 제공한다.

## 본문: 리스트 컴프리헨션 완전 정복

### 기본 문법

리스트 컴프리헨션의 기본 형태는 다음과 같다.

```python
[표현식 for 항목 in 반복가능객체]
```

앞서 본 제곱수 예제를 리스트 컴프리헨션으로 작성하면 이렇게 된다.

```python
squares = [i ** 2 for i in range(1, 11)]
print(squares)  # [1, 4, 9, 16, 25, 36, 49, 64, 81, 100]
```

단 한 줄로 같은 결과를 얻을 수 있다. 코드가 간결해졌을 뿐만 아니라 의도도 더 명확하게 드러난다.

### 조건문 추가하기

리스트 컴프리헨션에는 조건을 추가하여 특정 항목만 선택할 수 있다.

```python
[표현식 for 항목 in 반복가능객체 if 조건]
```

짝수만 골라서 제곱하는 예제를 보자.

```python
even_squares = [i ** 2 for i in range(1, 11) if i % 2 == 0]
print(even_squares)  # [4, 16, 36, 64, 100]
```

전통적인

## 1-3. Class 기반 구현 (Object-Oriented Approach)

재사용성과 유지보수성을 높이기 위해 클래스로 Skill 시스템을 구현하는 방법입니다.

In [11]:
from dataclasses import dataclass
from typing import Optional, List, Dict

@dataclass
class Skill:
    """
    Skill 정보를 담는 데이터 클래스
    
    Attributes:
        name: Skill의 이름
        description: Skill에 대한 간단한 설명
        content: Skill의 실제 내용 (프롬프트)
        version: Skill의 버전 정보
    """
    name: str
    description: str
    content: str
    version: str = "1.0.0"

In [13]:
class SkillAgent:
    """
    Skill을 관리하고 Claude API와 상호작용하는 에이전트 클래스
    """
    
    def __init__(self, api_key: Optional[str] = None):
        """
        SkillAgent를 초기화한다
        
        Args:
            api_key: Anthropic API 키 (None이면 환경 변수에서 가져온다)
        """
        self.client = anthropic.Anthropic(
            api_key=api_key or os.environ.get("ANTHROPIC_API_KEY")
        )
        self.skills: Dict[str, Skill] = {}  # Skill을 저장하는 딕셔너리
        
    def register_skill(self, skill: Skill) -> None:
        """
        새로운 Skill을 등록한다
        
        Args:
            skill: 등록할 Skill 객체
        """
        self.skills[skill.name] = skill
        print(f"Skill '{skill.name}' (v{skill.version})이 등록되었다")
    
    def list_skills(self) -> List[str]:
        """
        등록된 모든 Skill의 목록을 반환한다
        
        Returns:
            Skill 이름 리스트
        """
        return list(self.skills.keys())
    
    def execute(
        self, 
        skill_name: str, 
        user_message: str,
        model: str = MODEL,
        max_tokens: int = 2000
    ) -> str:
        """
        특정 Skill을 사용하여 Claude API를 호출한다
        
        Args:
            skill_name: 사용할 Skill의 이름
            user_message: 사용자의 요청 메시지
            model: 사용할 Claude 모델
            max_tokens: 최대 토큰 수
            
        Returns:
            Claude의 응답 텍스트
            
        Raises:
            ValueError: 존재하지 않는 Skill을 요청한 경우
        """
        if skill_name not in self.skills:
            raise ValueError(f"Skill '{skill_name}'을 찾을 수 없다. 등록된 Skills: {self.list_skills()}")
        
        skill = self.skills[skill_name]
        
        # Skill을 시스템 프롬프트로 사용하여 API 호출
        response = self.client.messages.create(
            model=model,
            max_tokens=max_tokens,
            system=skill.content,
            messages=[
                {
                    "role": "user",
                    "content": user_message
                }
            ]
        )
        
        return response.content[0].text

In [14]:
# SkillAgent 인스턴스 생성
agent = SkillAgent()

# 블로그 작성 Skill 등록
blog_skill = Skill(
    name="blog_writer",
    description="기술 블로그 포스트 작성 전문가",
    content=BLOG_WRITING_SKILL,
    version="1.0.0"
)
agent.register_skill(blog_skill)

# 코드 리뷰 Skill 생성 및 등록
code_review_skill = Skill(
    name="code_reviewer",
    description="코드 리뷰 및 개선 제안 전문가",
    content="""
# 코드 리뷰어 Skill

## 역할
제공된 코드를 분석하고 개선 방안을 제시한다.

## 리뷰 기준
1. 코드 가독성
2. 성능 최적화
3. 보안 취약점
4. 베스트 프랙티스 준수
5. 에러 처리

## 출력 형식
- 긍정적인 부분 먼저 언급한다
- 개선이 필요한 부분을 구체적으로 설명한다
- 개선된 코드 예시를 제공한다
- 우선순위를 명시한다
    """,
    version="1.0.0"
)
agent.register_skill(code_review_skill)

Skill 'blog_writer' (v1.0.0)이 등록되었다
Skill 'code_reviewer' (v1.0.0)이 등록되었다


In [15]:
# 등록된 Skill 목록 확인
print("\n등록된 Skills:")
for skill_name in agent.list_skills():
    skill = agent.skills[skill_name]
    print(f"  - {skill_name}: {skill.description}")


등록된 Skills:
  - blog_writer: 기술 블로그 포스트 작성 전문가
  - code_reviewer: 코드 리뷰 및 개선 제안 전문가


In [16]:
# Skill 실행 테스트
print("\n" + "=" * 80)
print("블로그 작성 Skill 테스트")
print("=" * 80)
result1 = agent.execute(
    skill_name="blog_writer",
    user_message="딕셔너리와 세트의 차이점을 설명하는 짧은 글을 써줘"
)
print(result1)


블로그 작성 Skill 테스트
# 파이썬 딕셔너리와 세트: 핵심 차이점 이해하기

## 비슷해 보이지만 다른 두 자료구조

파이썬을 사용하다 보면 딕셔너리(Dictionary)와 세트(Set)가 비슷해 보인다고 느낄 수 있다. 둘 다 중괄호 `{}`를 사용하고, 내부적으로 해시 테이블을 기반으로 동작하기 때문이다. 하지만 이 둘은 분명히 다른 목적을 가진 자료구조다.

## 핵심 차이점

### 1. 저장하는 데이터의 형태

**딕셔너리**는 키(key)와 값(value)의 **쌍**을 저장한다.

```python
user = {
    "name": "홍길동",
    "age": 25,
    "email": "hong@example.com"
}
```

**세트**는 **값만** 저장한다.

```python
fruits = {"사과", "바나나", "오렌지"}
```

### 2. 데이터 접근 방식

**딕셔너리**는 키를 통해 값에 접근한다.

```python
print(user["name"])  # "홍길동"
print(user["age"])   # 25
```

**세트**는 인덱스나 키로 접근할 수 없다. 멤버십 확인만 가능하다.

```python
print("사과" in fruits)  # True
# fruits[0]  # 에러 발생!
```

### 3. 사용 목적

**딕셔너리**는 데이터를 의미 있는 이름(키)으로 관리할 때 사용한다.
- 설정 정보 저장
- 데이터베이스 레코드 표현
- JSON 데이터 처리

```python
config = {
    "host": "localhost",
    "port": 8080,
    "debug": True
}
```

**세트**는 중복 제거와 집합 연산이 필요할 때 사용한다.
- 중복 데이터 제거
- 교집합, 합집합 등 집합 연산
- 빠른 멤버십 테스트

```python
numbers = [1, 2, 2, 3, 3, 3, 4]
unique_numbers = set(numbers)

In [17]:
print("\n" + "=" * 80)
print("코드 리뷰 Skill 테스트")
print("=" * 80)
code_to_review = """
def calc(a, b):
    return a + b

result = calc(5, 10)
print(result)
"""
result2 = agent.execute(
    skill_name="code_reviewer",
    user_message=f"다음 코드를 리뷰해줘:\n\n{code_to_review}"
)
print(result2)


코드 리뷰 Skill 테스트
# 코드 리뷰 결과

## ✅ 긍정적인 부분
1. **간결성**: 코드가 매우 간결하고 이해하기 쉽습니다
2. **동작 확인**: 기본적인 덧셈 기능이 정상적으로 작동합니다
3. **함수 분리**: 계산 로직을 함수로 분리한 점은 좋습니다

---

## 🔧 개선이 필요한 부분

### 1. **함수명 및 변수명 가독성** (우선순위: 높음)
- `calc`는 너무 일반적인 이름입니다
- 함수가 무엇을 하는지 명확하지 않습니다

### 2. **타입 힌트 부재** (우선순위: 높음)
- 입력 파라미터와 반환값의 타입이 명시되지 않았습니다
- 코드 유지보수성과 IDE 지원이 제한됩니다

### 3. **문서화 부재** (우선순위: 중간)
- Docstring이 없어 함수의 목적을 파악하기 어렵습니다

### 4. **에러 처리 부재** (우선순위: 중간)
- 잘못된 타입 입력에 대한 처리가 없습니다
- 예: `calc("5", 10)` 실행 시 예상치 못한 결과 발생

### 5. **테스트 코드 부재** (우선순위: 낮음)
- 단위 테스트가 없습니다

---

## 💡 개선된 코드 예시

```python
def add_numbers(num1: float, num2: float) -> float:
    """
    두 숫자를 더한 결과를 반환합니다.
    
    Args:
        num1 (float): 첫 번째 숫자
        num2 (float): 두 번째 숫자
    
    Returns:
        float: 두 숫자의 합
        
    Raises:
        TypeError: 입력값이 숫자가 아닌 경우
        
    Examples:
        >>> add_numbers(5, 10)
        15
        >>> add_numbers(3.5, 2.5)
        6.0
    """
    if not isinstance(num1, (int, floa

## 1-4. 실습

실습: 자신만의 Skill을 만들어보자

In [ ]:
# TODO: 아래 템플릿을 참고하여 새로운 Skill을 작성한다
custom_skill = Skill(
    name="your_skill_name",  # Skill 이름을 정한다
    description="당신의 Skill 설명",  # Skill이 하는 일을 설명한다
    content="""
# 당신의 Skill 제목

## 목적
이 Skill의 목적을 작성한다

## 원칙
1. 첫 번째 원칙
2. 두 번째 원칙

## 스타일
- 스타일 가이드를 작성한다
    """,
    version="1.0.0"
)

# Skill을 등록하고 테스트해본다
agent.register_skill(custom_skill)
result = agent.execute("your_skill_name", "테스트 메시지")
print(result)